<a href="https://colab.research.google.com/github/zizaixiongying/rs/blob/main/chengxu8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Copyright 2020 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/TF_demo1_keras.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/TF_demo1_keras.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Introduction

This is an Earth Engine <> TensorFlow demonstration notebook.  Specifically, this notebook shows:

1.   Exporting training/testing data from Earth Engine in TFRecord format.
2.   Preparing the data for use in a TensorFlow model.
2.   Training and validating a simple model (Keras `Sequential` neural network) in TensorFlow.
3.   Making predictions on image data exported from Earth Engine in TFRecord format.
4.   Ingesting classified image data to Earth Engine in TFRecord format.

This is intended to demonstrate a complete i/o pipeline.  For a worflow that uses a [Google AI Platform](https://cloud.google.com/ai-platform) hosted model making predictions interactively, see [this example notebook](http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_TensorFlow_AI_Platform.ipynb).

# Setup software libraries

Import software libraries and/or authenticate as necessary.

## Authenticate to Colab and Cloud

To read/write from a Google Cloud Storage bucket to which you have access, it's necessary to authenticate (as yourself).  *This should be the same account you use to login to Earth Engine*.  When you run the code below, it will display a link in the output to an authentication page in your browser.  Follow the link to a page that will let you grant permission to the Cloud SDK to access your resources.  Copy the code from the permissions page back into this notebook and press return to complete the process.

(You may need to run this again if you get a credentials error later.)

In [ ]:
from google.colab import auth
auth.authenticate_user()

## Authenticate to Earth Engine

Authenticate to Earth Engine the same way you did to the Colab notebook.  Specifically, run the code to display a link to a permissions page.  This gives you access to your Earth Engine account.  *This should be the same account you used to login to Cloud previously*.  Copy the code from the Earth Engine permissions page back into the notebook and press return to complete the process.

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

## Test the TensorFlow installation

Import the TensorFlow library and check the version.

In [ ]:
import tensorflow as tf
print(tf.__version__)

## Test the Folium installation

We will use the Folium library for visualization.  Import the library and check the version.

In [ ]:
import folium
print(folium.__version__)

In [ ]:
from pprint import pprint

# Define variables

This set of global variables will be used throughout.  For this demo, you must have a Cloud Storage bucket into which you can write files.  ([learn more about creating Cloud Storage buckets](https://cloud.google.com/storage/docs/creating-buckets)).  You'll also need to specify your Earth Engine username, i.e. `users/USER_NAME` on the [Code Editor](https://code.earthengine.google.com/) Assets tab.

In [ ]:
# Your Earth Engine username.  This is used to import a classified image
# into your Earth Engine assets folder.
USER_NAME = 'jz99544'

# Cloud Storage bucket into which training, testing and prediction
# datasets will be written.  You must be able to write into this bucket.
OUTPUT_BUCKET = 'zhangjian'
# Use Landsat 8 surface reflectance data for predictors.
# Use these bands for prediction.
BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'B1_1', 'B2_1', 'B3_1', 'B4_1', 'B5_1', 'B6_1', 'B7_1', 'B8', 'B8A', 'B9', 'B10_1', 'B11_1', 'B12', 'B1_2', 'B2_2', 'B3_2', 'B4_2', 'B5_2', 'B6_2', 'B7_2', 'B10_2', 'B11_2', 'B1_3', 'B2_3', 'B3_3', 'B4_3', 'B5_3', 'B6_3', 'B7_3', 'B8_1', 'B8A_1', 'B9_1', 'B10_3', 'B11_3', 'B12_1', 'B1_4', 'B2_4', 'B3_4', 'B4_4', 'B5_4', 'B6_4', 'B7_4', 'B10_4', 'B11_4', 'B1_5', 'B2_5', 'B3_5', 'B4_5', 'B5_5', 'B6_5', 'B7_5', 'B8_2', 'B8A_2', 'B9_2', 'B10_5', 'B11_5', 'B12_2', 'B1_6', 'B2_6', 'B3_6', 'B4_6', 'B5_6', 'B6_6', 'B7_6', 'B10_6', 'B11_6', 'B1_7', 'B2_7', 'B3_7', 'B4_7', 'B5_7', 'B6_7', 'B7_7', 'B8_3', 'B8A_3', 'B9_3', 'B10_7', 'B11_7', 'B12_3']
BANDS1 = ['B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7','B10','B11']
BANDS2 = ['B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10','B11','B12']
# BANDS = ['B2_-3_-3', 'B2_-2_-3', 'B2_-1_-3', 'B2_0_-3', 'B2_1_-3', 'B2_2_-3', 'B2_3_-3', 'B2_-3_-2', 'B2_-2_-2', 'B2_-1_-2', 'B2_0_-2', 'B2_1_-2', 'B2_2_-2', 'B2_3_-2', 'B2_-3_-1', 'B2_-2_-1', 'B2_-1_-1', 'B2_0_-1', 'B2_1_-1', 'B2_2_-1', 'B2_3_-1', 'B2_-3_0', 'B2_-2_0', 'B2_-1_0', 'B2_0_0', 'B2_1_0', 'B2_2_0', 'B2_3_0', 'B2_-3_1', 'B2_-2_1', 'B2_-1_1', 'B2_0_1', 'B2_1_1', 'B2_2_1', 'B2_3_1', 'B2_-3_2', 'B2_-2_2', 'B2_-1_2', 'B2_0_2', 'B2_1_2', 'B2_2_2', 'B2_3_2', 'B2_-3_3', 'B2_-2_3', 'B2_-1_3', 'B2_0_3', 'B2_1_3', 'B2_2_3', 'B2_3_3', 'B3_-3_-3', 'B3_-2_-3', 'B3_-1_-3', 'B3_0_-3', 'B3_1_-3', 'B3_2_-3', 'B3_3_-3', 'B3_-3_-2', 'B3_-2_-2', 'B3_-1_-2', 'B3_0_-2', 'B3_1_-2', 'B3_2_-2', 'B3_3_-2', 'B3_-3_-1', 'B3_-2_-1', 'B3_-1_-1', 'B3_0_-1', 'B3_1_-1', 'B3_2_-1', 'B3_3_-1', 'B3_-3_0', 'B3_-2_0', 'B3_-1_0', 'B3_0_0', 'B3_1_0', 'B3_2_0', 'B3_3_0', 'B3_-3_1', 'B3_-2_1', 'B3_-1_1', 'B3_0_1', 'B3_1_1', 'B3_2_1', 'B3_3_1', 'B3_-3_2', 'B3_-2_2', 'B3_-1_2', 'B3_0_2', 'B3_1_2', 'B3_2_2', 'B3_3_2', 'B3_-3_3', 'B3_-2_3', 'B3_-1_3', 'B3_0_3', 'B3_1_3', 'B3_2_3', 'B3_3_3', 'B4_-3_-3', 'B4_-2_-3', 'B4_-1_-3', 'B4_0_-3', 'B4_1_-3', 'B4_2_-3', 'B4_3_-3', 'B4_-3_-2', 'B4_-2_-2', 'B4_-1_-2', 'B4_0_-2', 'B4_1_-2', 'B4_2_-2', 'B4_3_-2', 'B4_-3_-1', 'B4_-2_-1', 'B4_-1_-1', 'B4_0_-1', 'B4_1_-1', 'B4_2_-1', 'B4_3_-1', 'B4_-3_0', 'B4_-2_0', 'B4_-1_0', 'B4_0_0', 'B4_1_0', 'B4_2_0', 'B4_3_0', 'B4_-3_1', 'B4_-2_1', 'B4_-1_1', 'B4_0_1', 'B4_1_1', 'B4_2_1', 'B4_3_1', 'B4_-3_2', 'B4_-2_2', 'B4_-1_2', 'B4_0_2', 'B4_1_2', 'B4_2_2', 'B4_3_2', 'B4_-3_3', 'B4_-2_3', 'B4_-1_3', 'B4_0_3', 'B4_1_3', 'B4_2_3', 'B4_3_3', 'B5_-3_-3', 'B5_-2_-3', 'B5_-1_-3', 'B5_0_-3', 'B5_1_-3', 'B5_2_-3', 'B5_3_-3', 'B5_-3_-2', 'B5_-2_-2', 'B5_-1_-2', 'B5_0_-2', 'B5_1_-2', 'B5_2_-2', 'B5_3_-2', 'B5_-3_-1', 'B5_-2_-1', 'B5_-1_-1', 'B5_0_-1', 'B5_1_-1', 'B5_2_-1', 'B5_3_-1', 'B5_-3_0', 'B5_-2_0', 'B5_-1_0', 'B5_0_0', 'B5_1_0', 'B5_2_0', 'B5_3_0', 'B5_-3_1', 'B5_-2_1', 'B5_-1_1', 'B5_0_1', 'B5_1_1', 'B5_2_1', 'B5_3_1', 'B5_-3_2', 'B5_-2_2', 'B5_-1_2', 'B5_0_2', 'B5_1_2', 'B5_2_2', 'B5_3_2', 'B5_-3_3', 'B5_-2_3', 'B5_-1_3', 'B5_0_3', 'B5_1_3', 'B5_2_3', 'B5_3_3', 'B6_-3_-3', 'B6_-2_-3', 'B6_-1_-3', 'B6_0_-3', 'B6_1_-3', 'B6_2_-3', 'B6_3_-3', 'B6_-3_-2', 'B6_-2_-2', 'B6_-1_-2', 'B6_0_-2', 'B6_1_-2', 'B6_2_-2', 'B6_3_-2', 'B6_-3_-1', 'B6_-2_-1', 'B6_-1_-1', 'B6_0_-1', 'B6_1_-1', 'B6_2_-1', 'B6_3_-1', 'B6_-3_0', 'B6_-2_0', 'B6_-1_0', 'B6_0_0', 'B6_1_0', 'B6_2_0', 'B6_3_0', 'B6_-3_1', 'B6_-2_1', 'B6_-1_1', 'B6_0_1', 'B6_1_1', 'B6_2_1', 'B6_3_1', 'B6_-3_2', 'B6_-2_2', 'B6_-1_2', 'B6_0_2', 'B6_1_2', 'B6_2_2', 'B6_3_2', 'B6_-3_3', 'B6_-2_3', 'B6_-1_3', 'B6_0_3', 'B6_1_3', 'B6_2_3', 'B6_3_3', 'B7_-3_-3', 'B7_-2_-3', 'B7_-1_-3', 'B7_0_-3', 'B7_1_-3', 'B7_2_-3', 'B7_3_-3', 'B7_-3_-2', 'B7_-2_-2', 'B7_-1_-2', 'B7_0_-2', 'B7_1_-2', 'B7_2_-2', 'B7_3_-2', 'B7_-3_-1', 'B7_-2_-1', 'B7_-1_-1', 'B7_0_-1', 'B7_1_-1', 'B7_2_-1', 'B7_3_-1', 'B7_-3_0', 'B7_-2_0', 'B7_-1_0', 'B7_0_0', 'B7_1_0', 'B7_2_0', 'B7_3_0', 'B7_-3_1', 'B7_-2_1', 'B7_-1_1', 'B7_0_1', 'B7_1_1', 'B7_2_1', 'B7_3_1', 'B7_-3_2', 'B7_-2_2', 'B7_-1_2', 'B7_0_2', 'B7_1_2', 'B7_2_2', 'B7_3_2', 'B7_-3_3', 'B7_-2_3', 'B7_-1_3', 'B7_0_3', 'B7_1_3', 'B7_2_3', 'B7_3_3']
# BANDS = ['B21', 'B22', 'B23', 'B24', 'B25', 'B26', 'B27', 'B28', 'B29',
#          'B3_1', 'B32', 'B33', 'B34', 'B35', 'B36', 'B37', 'B38', 'B39',
#          'B41', 'B42', 'B43', 'B44', 'B45', 'B46', 'B47', 'B48', 'B49',
#          'B51', 'B52', 'B53', 'B54', 'B55', 'B56', 'B57', 'B58', 'B59',
#          'B61', 'B62', 'B63', 'B64', 'B65', 'B66', 'B67', 'B68', 'B69',
#          'B71', 'B72', 'B73', 'B74', 'B75', 'B76', 'B77', 'B78', 'B79']

# This is a trianing/testing dataset of points with known land cover labels.
# LABEL_DATA = ee.FeatureCollection('projects/google/demo_landcover_labels')
# The labels, consecutive integer indices starting from zero, are stored in
# this property, set on each point.
LABEL = 'landcover'
# Number of label values, i.e. number of classes in the classification.
N_CLASSES = 14

# These names are used to specify properties in the export of
# training/testing data and to define the mapping between names and data
# when reading into TensorFlow datasets.
FEATURE_NAMES = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'B1_1', 'B2_1', 'B3_1', 'B4_1', 'B5_1', 'B6_1', 'B7_1', 'B8', 'B8A', 'B9', 'B10_1', 'B11_1', 'B12', 'B1_2', 'B2_2', 'B3_2', 'B4_2', 'B5_2', 'B6_2', 'B7_2', 'B10_2', 'B11_2', 'B1_3', 'B2_3', 'B3_3', 'B4_3', 'B5_3', 'B6_3', 'B7_3', 'B8_1', 'B8A_1', 'B9_1', 'B10_3', 'B11_3', 'B12_1', 'B1_4', 'B2_4', 'B3_4', 'B4_4', 'B5_4', 'B6_4', 'B7_4', 'B10_4', 'B11_4', 'B1_5', 'B2_5', 'B3_5', 'B4_5', 'B5_5', 'B6_5', 'B7_5', 'B8_2', 'B8A_2', 'B9_2', 'B10_5', 'B11_5', 'B12_2', 'B1_6', 'B2_6', 'B3_6', 'B4_6', 'B5_6', 'B6_6', 'B7_6', 'B10_6', 'B11_6', 'B1_7', 'B2_7', 'B3_7', 'B4_7', 'B5_7', 'B6_7', 'B7_7', 'B8_3', 'B8A_3', 'B9_3', 'B10_7', 'B11_7', 'B12_3']
# FEATURE_NAMES = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'B1_1', 'B2_1', 'B3_1', 'B4_1', 'B5_1', 'B6_1', 'B7_1', 'B10_1', 'B11_1', 'B1_2', 'B2_2', 'B3_2', 'B4_2', 'B5_2', 'B6_2', 'B7_2', 'B10_2', 'B11_2', 'B1_3', 'B2_3', 'B3_3', 'B4_3', 'B5_3', 'B6_3', 'B7_3', 'B10_3', 'B11_3']
FEATURE_NAMES.append(LABEL)

# File names for the training and testing datasets.  These TFRecord files
# will be exported from Earth Engine into the Cloud Storage bucket.
TRAIN_FILE_PREFIX = 'Training_demo'
TEST_FILE_PREFIX = 'Testing_demo'
file_extension = '.tfrecord.gz'
TRAIN_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TRAIN_FILE_PREFIX + file_extension
TEST_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TEST_FILE_PREFIX + file_extension

# File name for the prediction (image) dataset.  The trained model will read
# this dataset and make predictions in each pixel.
IMAGE_FILE_PREFIX = 'Image_pixel_demo_'

# The output path for the classified image (i.e. predictions) TFRecord file.
OUTPUT_IMAGE_FILE = 'gs://' + OUTPUT_BUCKET + '/Classified_pixel_demo1.TFRecord'
# Export imagery in this region.
# EXPORT_REGION = ee.Geometry.Rectangle([-83.32, 36.18, -83.02, 35.93])
# EXPORT_REGION = ee.Geometry.Rectangle([-96.67, 36.53,-96.37, 36.28])
EXPORT_REGION = ee.Geometry.Rectangle([-81.37, 26.14, -81.07, 25.89])


# The name of the Earth Engine asset to be created by importing
# the classified image from the TFRecord file in Cloud Storage.
OUTPUT_ASSET_ID = 'users/' + USER_NAME + '/Classified_pixel_demo1'

# Get Training and Testing data from Earth Engine

To get data for a classification model of three classes (bare, vegetation, water), we need labels and the value of predictor variables for each labeled example.  We've already generated some labels in Earth Engine.  Specifically, these are visually interpreted points labeled "bare," "vegetation," or "water" for a very simple classification demo ([example script](https://code.earthengine.google.com/?scriptPath=Examples%3ADemos%2FClassification)).  For predictor variables, we'll use [Landsat 8 surface reflectance imagery](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR), bands 2-7.

## Prepare Landsat 8 imagery

First, make a cloud-masked median composite of Landsat 8 surface reflectance imagery from 2018.  Check the composite by visualizing with folium.

In [ ]:
# Cloud masking function.
L8SR = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
# Cloud masking function.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).divide(10000)

S2 = ee.ImageCollection('COPERNICUS/S2')
def maskS2clouds(image):
  qa = image.select('QA60');
  cloudBitMask = ee.Number(2).pow(10).int()
  cirrusBitMask = ee.Number(2).pow(11).int()
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
      qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).divide(10000);

# The image input data is a cloud-masked median composite.
image1 = L8SR.filterDate('2016-04-01', '2016-08-31').map(maskL8sr).mean()
image2 = L8SR.filterDate('2016-05-01', '2016-09-30').map(maskL8sr).mean()
image3 = L8SR.filterDate('2016-06-01', '2016-10-31').map(maskL8sr).mean()
image4 = L8SR.filterDate('2016-07-01', '2016-11-30').map(maskL8sr).mean()
image5 = S2.filterDate('2016-04-01', '2016-08-31').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).map(maskS2clouds).mean()
image6 = S2.filterDate('2016-05-01', '2016-09-30').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).map(maskS2clouds).mean()
image7 = S2.filterDate('2016-06-01', '2016-10-31').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).map(maskS2clouds).mean()
image8 = S2.filterDate('2016-07-01', '2016-11-30').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).map(maskS2clouds).mean()

# list1 = ee.List.repeat(1, 7)
# lists1 = ee.List.repeat(list1, 7)
# kernel = ee.Kernel.fixed(7, 7, lists1)
# image = image.neighborhoodToBands(kernel)  #将邻域像元加入band
# image = image.select(['B2_-1_-1', 'B2_0_-1', 'B2_1_-1', 'B2_-1_0', 'B2_0_0', 'B2_1_0', 'B2_-1_1', 'B2_0_1', 'B2_1_1', 'B3_-1_-1', 'B3_0_-1', 'B3_1_-1', 'B3_-1_0', 'B3_0_0', 'B3_1_0', 'B3_-1_1', 'B3_0_1', 'B3_1_1', 'B4_-1_-1', 'B4_0_-1', 'B4_1_-1', 'B4_-1_0', 'B4_0_0', 'B4_1_0', 'B4_-1_1', 'B4_0_1', 'B4_1_1', 'B5_-1_-1', 'B5_0_-1', 'B5_1_-1', 'B5_-1_0', 'B5_0_0', 'B5_1_0', 'B5_-1_1', 'B5_0_1', 'B5_1_1', 'B6_-1_-1', 'B6_0_-1', 'B6_1_-1', 'B6_-1_0', 'B6_0_0', 'B6_1_0', 'B6_-1_1', 'B6_0_1', 'B6_1_1', 'B7_-1_-1', 'B7_0_-1', 'B7_1_-1', 'B7_-1_0', 'B7_0_0', 'B7_1_0', 'B7_-1_1', 'B7_0_1', 'B7_1_1'],
#          ['B21', 'B22', 'B23', 'B24', 'B25', 'B26', 'B27', 'B28', 'B29',
#          'B3_1', 'B32', 'B33', 'B34', 'B35', 'B36', 'B37', 'B38', 'B39',
#          'B41', 'B42', 'B43', 'B44', 'B45', 'B46', 'B47', 'B48', 'B49',
#          'B51', 'B52', 'B53', 'B54', 'B55', 'B56', 'B57', 'B58', 'B59',
#          'B61', 'B62', 'B63', 'B64', 'B65', 'B66', 'B67', 'B68', 'B69',
#          'B71', 'B72', 'B73', 'B74', 'B75', 'B76', 'B77', 'B78', 'B79'])
# print(image.bandNames().getInfo())
# # Use folium to visualize the imagery.
# mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
# # map = folium.Map(location=[38., -122.5])
# map = folium.Map(location=[37., 111.0])

# folium.TileLayer(
#     tiles=mapid['tile_fetcher'].url_format,
#     attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
#     overlay=True,
#     name='median composite',
#   ).add_to(map)
# map.add_child(folium.LayerControl())
# map

In [ ]:
nlcd = ee.Image('USGS/NLCD/NLCD2016').select('landcover')
# nlcd = nlcd.remap([11,12,21,22,23,24,31,41,42,43,51,52,71,72,73,74,81,82,90,95],[0,20,1,20,2,2,20,3,4,20,20,20,5,20,20,20,6,7,20,20])

# nlcd = nlcd.remap([11,12,21,22,23,24,31,41,42,43,51,52,71,72,73,74,81,82,90,95],[0,20,1,1,1,1,2,3,3,3,20,4,5,20,20,20,6,7,8,8])
# nlcd = nlcd.remap([11,12,21,22,23,24,31,41,42,43,51,52,71,72,73,74,81,82,90,95],[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])
nlcd = nlcd.remap([11,12,21,22,23,24,31,41,42,43,51,52,71,72,73,74,81,82,90,95],[0,20,1,2,3,3,4,5,6,7,20,8,9,20,20,20,10,11,12,13])
# nlcd = nlcd.remap([11,12,21,22,23,24,31,41,42,43,51,52,71,72,73,74,81,82,90,95],[0,9,1,9,9,2,9,3,4,5,9,9,9,9,9,9,6,9,7,9])
# nlcd = nlcd.remap([11,12,21,22,23,24,31,41,42,43,51,52,71,72,73,74,81,82,90,95],[0,20,1,1,2,2,20,3,4,4,20,4,4,20,20,20,5,5,4,0])
nlcd = nlcd.select(['remapped'],['landcover']) #nlcd在执行remap后landcover变成remaped，通过该方法将其再转换为landcover
image = ee.Image.cat([
  image1.select(BANDS1),
  image5.select(BANDS2),
  image2.select(BANDS1),
  image6.select(BANDS2),
  image3.select(BANDS1),
  image7.select(BANDS2),
  image4.select(BANDS1),
  image8.select(BANDS2),
]).float()
ln = ee.Image.cat([
  image,
  nlcd
]).float()

print(ln.bandNames().getInfo())

In [ ]:
poly1 = ee.Geometry.Polygon(
        [[[-90.24, 29.97],
          [-90.24, 29.27],
          [-89.24, 29.27],
          [-89.24, 29.97]]])
poly2 = ee.Geometry.Polygon(
        [[[-95.90, 30.20],
          [-95.90, 29.50],
          [-94.90, 29.50],
          [-94.90, 30.20]]])
poly3 = ee.Geometry.Polygon(
        [[[-84.78, 34.20],
          [-84.78, 33.50],
          [-83.78, 33.50],
          [-83.78, 34.20]]])
poly4 = ee.Geometry.Polygon(
        [[[-97.37, 33.31],
          [-97.37, 32.61],
          [-96.37, 32.61],
          [-96.37, 33.31]]])
poly5 = ee.Geometry.Polygon(
        [[[-116.52, 33.44],
          [-116.52, 32.74],
          [-115.52, 32.74],
          [-115.52, 33.44]]])
poly6 = ee.Geometry.Polygon(
        [[[-82.87, 38.05],
          [-82.87, 37.35],
          [-81.87, 37.35],
          [-81.87, 38.05]]])
poly7 = ee.Geometry.Polygon(
        [[[-104.32, 44.48],
          [-104.32, 43.78],
          [-103.32, 43.78],
          [-103.32, 44.48]]])
poly8 = ee.Geometry.Polygon(
        [[[-70.12, 44.74],
          [-70.12, 44.04],
          [-69.12, 44.04],
          [-69.12, 44.74]]])
poly9 = ee.Geometry.Polygon(
        [[[-103.61, 31.79],
          [-103.61, 30.79],
          [-102.61, 30.79],
          [-102.61, 31.79]]])
poly10 = ee.Geometry.Polygon(
        [[[-101.86, 42.48],
          [-101.86, 41.78],
          [-100.86, 41.78],
          [-100.86, 42.48]]])
poly11 = ee.Geometry.Polygon(
        [[[-85.07, 38.50],
          [-85.07, 37.80],
          [-84.07, 37.80],
          [-84.07, 38.50]]])
poly12 = ee.Geometry.Polygon(
        [[[-87.79, 41.92],
          [-87.79, 41.22],
          [-86.79, 41.22],
          [-86.79, 41.92]]])
poly13 = ee.Geometry.Polygon(
        [[[-94.54, 48.43],
          [-94.54, 47.73],
          [-93.54, 47.73],
          [-93.54, 48.43]]])

percent = 0.7
num = 5000
li = 500
sample1 = ln.sample(
    region = poly1,
    scale = 30,
    numPixels = num, # Size of the shard.
    seed = 3,
    tileScale = 8
  )
sample2 = ln.sample(
    region = poly2,
    scale = 30,
    numPixels = num, # Size of the shard.
    seed = 3,
    tileScale = 8
  )
sample3 = ln.sample(
    region = poly3,
    scale = 30,
    numPixels = num, # Size of the shard.
    seed = 3,
    tileScale = 8
  )
sample4 = ln.sample(
    region = poly4,
    scale = 30,
    numPixels = num, # Size of the shard.
    seed = 3,
    tileScale = 8
  )
sample5 = ln.sample(
    region = poly5,
    scale = 30,
    numPixels = num, # Size of the shard.
    seed = 3,
    tileScale = 8
  )
sample6 = ln.sample(
    region = poly6,
    scale = 30,
    numPixels = num, # Size of the shard.
    seed = 3,
    tileScale = 8
  )
sample7 = ln.sample(
    region = poly7,
    scale = 30,
    numPixels = num, # Size of the shard.
    seed = 3,
    tileScale = 8
  )
sample8 = ln.sample(
    region = poly8,
    scale = 30,
    numPixels = num, # Size of the shard.
    seed = 3,
    tileScale = 8
  )
sample9 = ln.sample(
    region = poly9,
    scale = 30,
    numPixels = num, # Size of the shard.
    seed = 3,
    tileScale = 8
  )
sample10 = ln.sample(
    region = poly10,
    scale = 30,
    numPixels = num, # Size of the shard.
    seed = 3,
    tileScale = 8
  )
sample11 = ln.sample(
    region = poly11,
    scale = 30,
    numPixels = num, # Size of the shard.
    seed = 3,
    tileScale = 8
  )
sample12 = ln.sample(
    region = poly12,
    scale = 30,
    numPixels = num, # Size of the shard.
    seed = 3,
    tileScale = 8
  )
sample13 = ln.sample(
    region = poly13,
    scale = 30,
    numPixels = num, # Size of the shard.
    seed = 3,
    tileScale = 8
  )

sample = sample1.merge(sample2).merge(sample3).merge(sample4).merge(sample5).merge(sample6).merge(sample7).merge(sample8).merge(sample9).merge(sample10).merge(sample11).merge(sample12).merge(sample13)
# sample = sample2.merge(sample3).merge(sample4).merge(sample5).merge(sample6).merge(sample7)
# for i in range(14):
#   a0 = sample.filter(ee.Filter.eq('landcover',i))
#   pprint("a0:%d"%(a0.size().getInfo()))

a0 = sample.filter(ee.Filter.eq('landcover',0)).limit(li).randomColumn()
b0 = a0.filter(ee.Filter.lt('random',percent))
c0 = a0.filter(ee.Filter.gte('random',percent))
a1 = sample.filter(ee.Filter.eq('landcover',1)).limit(li).randomColumn()
b1 = a1.filter(ee.Filter.lt('random',percent))
c1 = a1.filter(ee.Filter.gte('random',percent))
a2 = sample.filter(ee.Filter.eq('landcover',2)).limit(li).randomColumn()
b2 = a2.filter(ee.Filter.lt('random',percent))
c2 = a2.filter(ee.Filter.gte('random',percent))
a3 = sample.filter(ee.Filter.eq('landcover',3)).limit(li).randomColumn()
b3 = a3.filter(ee.Filter.lt('random',percent))
c3 = a3.filter(ee.Filter.gte('random',percent))
a4 = sample.filter(ee.Filter.eq('landcover',4)).limit(li).randomColumn()
b4 = a4.filter(ee.Filter.lt('random',percent))
c4 = a4.filter(ee.Filter.gte('random',percent))
a5 = sample.filter(ee.Filter.eq('landcover',5)).limit(li).randomColumn()
b5 = a5.filter(ee.Filter.lt('random',percent))
c5 = a5.filter(ee.Filter.gte('random',percent))
a6 = sample.filter(ee.Filter.eq('landcover',6)).limit(li).randomColumn()
b6 = a6.filter(ee.Filter.lt('random',percent))
c6 = a6.filter(ee.Filter.gte('random',percent))
a7 = sample.filter(ee.Filter.eq('landcover',7)).limit(li).randomColumn()
b7 = a7.filter(ee.Filter.lt('random',percent))
c7 = a7.filter(ee.Filter.gte('random',percent))
a8 = sample.filter(ee.Filter.eq('landcover',8)).limit(li).randomColumn()
b8 = a8.filter(ee.Filter.lt('random',percent))
c8 = a8.filter(ee.Filter.gte('random',percent))
a9 = sample.filter(ee.Filter.eq('landcover',9)).limit(li).randomColumn()
b9 = a9.filter(ee.Filter.lt('random',percent))
c9 = a9.filter(ee.Filter.gte('random',percent))
a10 = sample.filter(ee.Filter.eq('landcover',10)).limit(li).randomColumn()
b10 = a10.filter(ee.Filter.lt('random',percent))
c10 = a10.filter(ee.Filter.gte('random',percent))
a11 = sample.filter(ee.Filter.eq('landcover',11)).limit(li).randomColumn()
b11 = a11.filter(ee.Filter.lt('random',percent))
c11 = a11.filter(ee.Filter.gte('random',percent))
a12 = sample.filter(ee.Filter.eq('landcover',12)).limit(li).randomColumn()
b12 = a12.filter(ee.Filter.lt('random',percent))
c12 = a12.filter(ee.Filter.gte('random',percent))
a13 = sample.filter(ee.Filter.eq('landcover',13)).limit(li).randomColumn()
b13 = a13.filter(ee.Filter.lt('random',percent))
c13 = a13.filter(ee.Filter.gte('random',percent))
b = b0.merge(b1).merge(b2).merge(b3).merge(b4).merge(b5).merge(b6).merge(b7).merge(b8).merge(b9).merge(b10).merge(b11).merge(b12).merge(b13)
c = c0.merge(c1).merge(c2).merge(c3).merge(c4).merge(c5).merge(c6).merge(c7).merge(c8).merge(c9).merge(c10).merge(c11).merge(c12).merge(c13)
training = b
testing = c
# for i in range(14):
  # a0 = sample.filter(ee.Filter.eq('landcover',i)).limit(li).randomColumn()
  # b0 = a0.filter(ee.Filter.lt('random',percent))
  # c0 = a0.filter(ee.Filter.gte('random',percent))
  # pprint("b0:%d"%(a0.size().getInfo()))
  # pprint("c0:%d"%(c0.size().getInfo()))

# pprint("a0:%d"%(a0.size().getInfo()))
# pprint("a1:%d"%(a1.size().getInfo()))
# pprint("a2:%d"%(a2.size().getInfo()))
# pprint("a3:%d"%(a3.size().getInfo()))
# pprint("a4:%d"%(a4.size().getInfo()))
# pprint("a5:%d"%(a5.size().getInfo()))
# pprint("a6:%d"%(a6.size().getInfo()))
# pprint("a7:%d"%(a7.size().getInfo()))
# pprint("a8:%d"%(a8.size().getInfo()))
# trainingPolys = ee.FeatureCollection('projects/google/DemoTrainingGeometries')
# trainingPolysList = trainingPolys.toList(trainingPolys.size())
# geomSample = ee.FeatureCollection([])
# for g in range(trainingPolys.size().getInfo()):
#   sample = ln.sample(
#     # region = ee.Feature(trainingPolysList.get(g)).geometry(),
#     region = label_data,
#     scale = 30,
#     numPixels = 1000, # Size of the shard.
#     seed = 3,
#     tileScale = 8
#   )
#   geomSample = geomSample.merge(sample)

## Add pixel values of the composite to labeled points

Some training labels have already been collected for you.  Load the labeled points from an existing Earth Engine asset.  Each point in this table has a property called `landcover` that stores the label, encoded as an integer.  Here we overlay the points on  imagery to get predictor variables along with labels.

In [ ]:
# Sample the image at the points and add a random column.
# sample = ln.sampleRegions(
#   collection=label_data, scale=30)
# sample = sample.randomColumn()
# sample = sample.sort('random')

# percent = 0.6
# li = 4000

# a0 = sample.filter(ee.Filter.eq('landcover',0)).limit(li).randomColumn()
# b0 = a0.filter(ee.Filter.lt('random',percent))
# c0 = a0.filter(ee.Filter.gte('random',percent))
# a1 = sample.filter(ee.Filter.eq('landcover',1)).limit(li).randomColumn()
# b1 = a1.filter(ee.Filter.lt('random',percent))
# c1 = a1.filter(ee.Filter.gte('random',percent))
# a2 = sample.filter(ee.Filter.eq('landcover',2)).limit(li).randomColumn()
# b2 = a2.filter(ee.Filter.lt('random',percent))
# c2 = a2.filter(ee.Filter.gte('random',percent))
# a3 = sample.filter(ee.Filter.eq('landcover',3)).limit(li).randomColumn()
# b3 = a3.filter(ee.Filter.lt('random',percent))
# c3 = a3.filter(ee.Filter.gte('random',percent))
# a4 = sample.filter(ee.Filter.eq('landcover',4)).limit(li).randomColumn()
# b4 = a4.filter(ee.Filter.lt('random',percent))
# c4 = a4.filter(ee.Filter.gte('random',percent))
# a5 = sample.filter(ee.Filter.eq('landcover',5)).limit(li).randomColumn()
# b5 = a5.filter(ee.Filter.lt('random',percent))
# c5 = a5.filter(ee.Filter.gte('random',percent))
# a6 = sample.filter(ee.Filter.eq('landcover',6)).limit(li).randomColumn()
# b6 = a6.filter(ee.Filter.lt('random',percent))
# c6 = a6.filter(ee.Filter.gte('random',percent))
# a7 = sample.filter(ee.Filter.eq('landcover',7)).limit(li).randomColumn()
# b7 = a7.filter(ee.Filter.lt('random',percent))
# c7 = a7.filter(ee.Filter.gte('random',percent))
# a8 = sample.filter(ee.Filter.eq('landcover',8)).limit(li).randomColumn()
# b8 = a8.filter(ee.Filter.lt('random',percent))
# c8 = a8.filter(ee.Filter.gte('random',percent))
# a9 = sample.filter(ee.Filter.eq('landcover',9)).limit(li).randomColumn()
# b9 = a9.filter(ee.Filter.lt('random',percent))
# c9 = a9.filter(ee.Filter.gte('random',percent))
# a10 = sample.filter(ee.Filter.eq('landcover',10)).limit(li).randomColumn()
# b10 = a10.filter(ee.Filter.lt('random',percent))
# c10 = a10.filter(ee.Filter.gte('random',percent))
# a11 = sample.filter(ee.Filter.eq('landcover',11)).limit(li).randomColumn()
# b11 = a11.filter(ee.Filter.lt('random',percent))
# c11 = a11.filter(ee.Filter.gte('random',percent))
# a12 = sample.filter(ee.Filter.eq('landcover',12)).limit(li).randomColumn()
# b12 = a12.filter(ee.Filter.lt('random',percent))
# c12 = a12.filter(ee.Filter.gte('random',percent))
# a13 = sample.filter(ee.Filter.eq('landcover',13)).limit(li).randomColumn()
# b13 = a13.filter(ee.Filter.lt('random',percent))
# c13 = a13.filter(ee.Filter.gte('random',percent))
# a14 = sample.filter(ee.Filter.eq('landcover',14)).limit(li).randomColumn()
# b14 = a14.filter(ee.Filter.lt('random',percent))
# c14 = a14.filter(ee.Filter.gte('random',percent))
# a15 = sample.filter(ee.Filter.eq('landcover',15)).limit(li).randomColumn()
# b15 = a15.filter(ee.Filter.lt('random',percent))
# c15 = a15.filter(ee.Filter.gte('random',percent))
# a16 = sample.filter(ee.Filter.eq('landcover',16)).limit(li).randomColumn()
# b16 = a16.filter(ee.Filter.lt('random',percent))
# c16 = a16.filter(ee.Filter.gte('random',percent))
# a17 = sample.filter(ee.Filter.eq('landcover',17)).limit(li).randomColumn()
# b17 = a17.filter(ee.Filter.lt('random',percent))
# c17 = a17.filter(ee.Filter.gte('random',percent))
# a18 = sample.filter(ee.Filter.eq('landcover',18)).limit(li).randomColumn()
# b18 = a18.filter(ee.Filter.lt('random',percent))
# c18 = a18.filter(ee.Filter.gte('random',percent))
# a19 = sample.filter(ee.Filter.eq('landcover',19)).limit(li).randomColumn()
# b19 = a19.filter(ee.Filter.lt('random',percent))
# c19 = a19.filter(ee.Filter.gte('random',percent))
# b = b0.merge(b1).merge(b2).merge(b3).merge(b4).merge(b5).merge(b6).merge(b7).merge(b8)
# c = c0.merge(c1).merge(c2).merge(c3).merge(c4).merge(c5).merge(c6).merge(c7).merge(c8)
# training = b
# testing = c
# # Partition the sample approximately 70-30.
# # training = sample.filter(ee.Filter.lt('random', 0.1))
# # testing = sample.filter(ee.Filter.gt('random', 0))
# # training = geomSample
# pprint("a0:%d"%(a0.size().getInfo()))
# pprint("a1:%d"%(a1.size().getInfo()))
# pprint("a2:%d"%(a2.size().getInfo()))
# pprint("a3:%d"%(a3.size().getInfo()))
# pprint("a4:%d"%(a4.size().getInfo()))
# pprint("a5:%d"%(a5.size().getInfo()))
# pprint("a6:%d"%(a6.size().getInfo()))
# pprint("a7:%d"%(a7.size().getInfo()))
# pprint("a8:%d"%(a8.size().getInfo()))
# pprint("a9:%d"%(a9.size().getInfo()))
# pprint("a10:%d"%(a10.size().getInfo()))
# pprint("a11:%d"%(a11.size().getInfo()))
# pprint("a12:%d"%(a12.size().getInfo()))
# pprint("a13:%d"%(a13.size().getInfo()))
# pprint("a14:%d"%(a14.size().getInfo()))
# pprint("a15:%d"%(a15.size().getInfo()))
# pprint("a16:%d"%(a16.size().getInfo()))
# pprint("a17:%d"%(a17.size().getInfo()))
# pprint("a18:%d"%(a18.size().getInfo()))
# pprint("a19:%d"%(a19.size().getInfo()))

## Export the training and testing data

Now that there's training and testing data in Earth Engine and you've inspected a couple examples to ensure that the information you need is present, it's time to materialize the datasets in a place where the TensorFlow model has access to them.  You can do that by exporting the training and testing datasets to tables in TFRecord format ([learn more about TFRecord format](https://www.tensorflow.org/tutorials/load_data/tf-records)) in your Cloud Storage bucket.

In [ ]:
# Make sure you can see the output bucket.  You must have write access.
print('Found Cloud Storage bucket.' if tf.io.gfile.exists('gs://' + OUTPUT_BUCKET)
    else 'Can not find output Cloud Storage bucket.')

Once you've verified the existence of the intended output bucket, run the exports.

In [ ]:
# Create the tasks.
training_task = ee.batch.Export.table.toCloudStorage(
  collection=training,
  description='Training Export',
  fileNamePrefix=TRAIN_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

testing_task = ee.batch.Export.table.toCloudStorage(
  collection=testing,
  description='Testing Export',
  fileNamePrefix=TEST_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

In [ ]:
# Start the tasks.
training_task.start()
testing_task.start()

### Monitor task progress

You can see all your Earth Engine tasks by listing them.  Make sure the training and testing tasks are completed before continuing.

In [ ]:
# Print all tasks.
pprint(ee.batch.Task.list())

### Check existence of the exported files

If you've seen the status of the export tasks change to `COMPLETED`, then check for the existince of the files in the output Cloud Storage bucket.

In [ ]:
print('Found training file.' if tf.io.gfile.exists(TRAIN_FILE_PATH)
    else 'No training file found.')
print('Found testing file.' if tf.io.gfile.exists(TEST_FILE_PATH)
    else 'No testing file found.')

## Export the imagery

You can also export imagery using TFRecord format.  Specifically, export whatever imagery you want to be classified by the trained model into the output Cloud Storage bucket.

In [ ]:
# Specify patch and file dimensions.
image_export_options = {
  'patchDimensions': [256,256],
  'maxFileSize': 104857600,
  'compressed': True
}

# Setup the task.
image_task = ee.batch.Export.image.toCloudStorage(
  image=image,
  description='Image Export',
  fileNamePrefix=IMAGE_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  scale=30,
  fileFormat='TFRecord',
  region=EXPORT_REGION.toGeoJSON()['coordinates'],
  formatOptions=image_export_options,
)

In [ ]:
# Start the task.
image_task.start()

### Monitor task progress

In [ ]:
# Print all tasks.
pprint(ee.batch.Task.list())

It's also possible to monitor an individual task.  Here we poll the task until it's done.  If you do this, please put a `sleep()` in the loop to avoid making too many requests.  Note that this will block until complete (you can always halt the execution of this cell).

In [ ]:
import time

while image_task.active():
  print('Polling for task (id: {}).'.format(image_task.id))
  time.sleep(30)
print('Done with image export.')

# Data preparation and pre-processing

Read data from the TFRecord file into a `tf.data.Dataset`.  Pre-process the dataset to get it into a suitable format for input to the model.

## Read into a `tf.data.Dataset`

Here we are going to read a file in Cloud Storage into a `tf.data.Dataset`.  ([these TensorFlow docs](https://www.tensorflow.org/guide/data) explain more about reading data into a `Dataset`).  Check that you can read examples from the file.  The purpose here is to ensure that we can read from the file without an error.  The actual content is not necessarily human readable.



In [ ]:
# Create a dataset from the TFRecord file in Cloud Storage.
train_dataset = tf.data.TFRecordDataset(TRAIN_FILE_PATH, compression_type='GZIP')
train_dataset = train_dataset.shuffle(50000,seed=1)
# Print the first record to check.
print(iter(train_dataset).next())

## Define the structure of your data

For parsing the exported TFRecord files, `featuresDict` is a mapping between feature names (recall that `featureNames` contains the band and label names) and `float32` [`tf.io.FixedLenFeature`](https://www.tensorflow.org/api_docs/python/tf/io/FixedLenFeature) objects.  This mapping is necessary for telling TensorFlow how to read data in a TFRecord file into tensors.  Specifically, **all numeric data exported from Earth Engine is exported as `float32`**.

(Note: *features* in the TensorFlow context (i.e. [`tf.train.Feature`](https://www.tensorflow.org/api_docs/python/tf/train/Feature)) are not to be confused with Earth Engine features (i.e. [`ee.Feature`](https://developers.google.com/earth-engine/api_docs#eefeature)), where the former is a protocol message type for serialized data input to the model and the latter is a geometry-based geographic data structure.)

In [ ]:
# List of fixed-length features, all of which are float32.
columns = [
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32) for k in FEATURE_NAMES
]

# Dictionary with names as keys, features as values.
features_dict = dict(zip(FEATURE_NAMES, columns))
pprint(features_dict)

## Parse the dataset

Now we need to make a parsing function for the data in the TFRecord files.  The data comes in flattened 2D arrays per record and we want to use the first part of the array for input to the model and the last element of the array as the class label.  The parsing function reads data from a serialized [`Example` proto](https://www.tensorflow.org/api_docs/python/tf/train/Example) into a dictionary in which the keys are the feature names and the values are the tensors storing the value of the features for that example.  ([These TensorFlow docs](https://www.tensorflow.org/tutorials/load_data/tfrecord) explain more about reading `Example` protos from TFRecord files).

In [ ]:
def parse_tfrecord(example_proto):
  """The parsing function.

  Read a serialized example into the structure defined by featuresDict.

  Args:
    example_proto: a serialized Example.

  Returns:
    A tuple of the predictors dictionary and the label, cast to an `int32`.
  """
  parsed_features = tf.io.parse_single_example(example_proto, features_dict)
  labels = parsed_features.pop(LABEL)
  return parsed_features, tf.cast(labels, tf.int32)

# Map the function over the dataset.
parsed_dataset = train_dataset.map(parse_tfrecord, num_parallel_calls=5)

# Print the first parsed record to check.
pprint(iter(parsed_dataset).next())

Note that each record of the parsed dataset contains a tuple.  The first element of the tuple is a dictionary with bands for keys and the numeric value of the bands for values.  The second element of the tuple is a class label.

## Create additional features

Another thing we might want to do as part of the input process is to create new features, for example NDVI, a vegetation index computed from reflectance in two spectral bands.  Here are some helper functions for that.

In [ ]:
def normalized_difference(a, b):
  """Compute normalized difference of two inputs.

  Compute (a - b) / (a + b).  If the denomenator is zero, add a small delta.

  Args:
    a: an input tensor with shape=[1]
    b: an input tensor with shape=[1]

  Returns:
    The normalized difference as a tensor.
  """
  nd = (a - b) / (a + b)
  nd_inf = (a - b) / (a + b + 0.000001)
  return tf.where(tf.math.is_finite(nd), nd, nd_inf)

def add_NDVI(features, label):
  """Add NDVI to the dataset.
  Args:
    features: a dictionary of input tensors keyed by feature name.
    label: the target label

  Returns:
    A tuple of the input dictionary with an NDVI tensor added and the label.
  """
  features['NDVI'] = normalized_difference(features['B5_0_0'], features['B4_0_0'])
  return features, label

# Model setup

The basic workflow for classification in TensorFlow is:

1.  Create the model.
2.  Train the model (i.e. `fit()`).
3.  Use the trained model for inference (i.e. `predict()`).

Here we'll create a `Sequential` neural network model using Keras.  This simple model is inspired by examples in:

* [The TensorFlow Get Started tutorial](https://www.tensorflow.org/tutorials/)
* [The TensorFlow Keras guide](https://www.tensorflow.org/guide/keras#build_a_simple_model)
* [The Keras `Sequential` model examples](https://keras.io/getting-started/sequential-model-guide/#multilayer-perceptron-mlp-for-multi-class-softmax-classification)

Note that the model used here is purely for demonstration purposes and hasn't gone through any performance tuning.

## Create the Keras model

Before we create the model, there's still a wee bit of pre-processing to get the data into the right input shape and a format that can be used with cross-entropy loss.  Specifically, Keras expects a list of inputs and a one-hot vector for the class. (See [the Keras loss function docs](https://keras.io/losses/), [the TensorFlow categorical identity docs](https://www.tensorflow.org/guide/feature_columns#categorical_identity_column) and [the `tf.one_hot` docs](https://www.tensorflow.org/api_docs/python/tf/one_hot) for details).  

Here we will use a simple neural network model with a 64 node hidden layer, a dropout layer and an output layer.  Once the dataset has been prepared, define the model, compile it, fit it to the training data.  See [the Keras `Sequential` model guide](https://keras.io/getting-started/sequential-model-guide/) for more details.

In [ ]:
from tensorflow import keras
from sklearn import preprocessing

# Add NDVI.
# input_dataset = parsed_dataset.map(add_NDVI)
# input_dataset = parsed_dataset

# Keras requires inputs as a tuple.  Note that the inputs must be in the
# right shape.  Also note that to use the categorical_crossentropy loss,
# the label needs to be turned into a one-hot vector.
def to_tuple(inputs, label):
  return (tf.transpose(list(inputs.values())),
          tf.one_hot(indices=label, depth=N_CLASSES))

# Map the to_tuple function, shuffle and batch.

input_dataset = parsed_dataset.map(to_tuple).batch(64)
# Define the layers in the model.
model = tf.keras.models.Sequential([
  # tf.keras.layers.Input(shape=(1,88)),
  # tf.keras.layers.BatchNormalization(),
  # tf.keras.layers.Dense(64, activation=tf.nn.relu),
  # tf.keras.layers.BatchNormalization(),
  # tf.keras.layers.Dense(64, activation=tf.nn.relu),
  # tf.keras.layers.BatchNormalization(),
  # tf.keras.layers.Dense(64, activation=tf.nn.relu),


  # tf.keras.layers.Input(shape=(1,108)),
  # tf.keras.layers.BatchNormalization(),
  # tf.keras.layers.Dense(32, activation=tf.nn.relu),
  # tf.keras.layers.BatchNormalization(),
  # tf.keras.layers.Dense(32, activation=tf.nn.relu),
  # tf.keras.layers.BatchNormalization(),
  # tf.keras.layers.Dense(32, activation=tf.nn.relu),
  # tf.keras.layers.BatchNormalization(),
  # tf.keras.layers.Dense(32, activation=tf.nn.relu,kernel_regularizer=keras.regularizers.l2(0.01)),

  # tf.keras.layers.Input(shape=(1,88)),
  # tf.keras.layers.BatchNormalization(),
  # tf.keras.layers.Reshape([4,22,1]),
  # tf.keras.layers.Conv2D(32, (2,4), strides=(1,1), padding='valid',activation=tf.nn.relu),
  # tf.keras.layers.Conv2D(32, (2,4), strides=(1,1), padding='valid',activation=tf.nn.relu),
  # tf.keras.layers.Conv2D(32, (2,4), strides=(1,1), padding='valid',activation=tf.nn.relu),
  # tf.keras.layers.Flatten(),
  # tf.keras.layers.Dense(100, activation='relu'),
  # tf.keras.layers.Reshape([1,100]),

  # tf.keras.layers.Input(shape=(1,88)),
  # tf.keras.layers.BatchNormalization(),
  # tf.keras.layers.Reshape([22,4,1]),
  # tf.keras.layers.Conv2D(32, (4,2), strides=(1,1), padding='valid',activation=tf.nn.relu),
  # tf.keras.layers.Conv2D(32, (4,2), strides=(1,1), padding='valid',activation=tf.nn.relu),
  # # tf.keras.layers.Conv2D(32, (4,2), strides=(1,1), padding='valid',activation=tf.nn.relu),
  # tf.keras.layers.Flatten(),
  # tf.keras.layers.Dense(100, activation='relu'),
  # tf.keras.layers.Reshape([1,100]),

  tf.keras.layers.Input(shape=(1,88)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Reshape([8,11,1]),
  tf.keras.layers.Conv2D(32, (2,2), strides=(1,1), padding='valid',activation=tf.nn.relu),
  tf.keras.layers.Conv2D(32, (2,2), strides=(1,1), padding='valid',activation=tf.nn.relu),
  tf.keras.layers.MaxPooling2D((2,2),strides=(2,2)),
  tf.keras.layers.Conv2D(32, (2,2), strides=(1,1), padding='valid',activation=tf.nn.relu),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Reshape([1,100]),

  # tf.keras.layers.Input(shape=(1,108)),
  # tf.keras.layers.BatchNormalization(),
  # tf.keras.layers.Reshape([12,9,1]),
  # tf.keras.layers.Conv2D(32, (3,3), strides=(1,1), padding='valid',activation=tf.nn.relu),
  # tf.keras.layers.Conv2D(32, (3,3), strides=(1,1), padding='valid',activation=tf.nn.relu),
  # tf.keras.layers.Conv2D(32, (3,3), strides=(1,1), padding='valid',activation=tf.nn.relu),
  # tf.keras.layers.Conv2D(32, (3,3), strides=(1,1), padding='valid',activation=tf.nn.relu),
  # tf.keras.layers.Flatten(),
  # # tf.keras.layers.Dense(32, activation='relu'),
  # # tf.keras.layers.Dropout(0.8),
  # tf.keras.layers.Reshape([1,128]),




  tf.keras.layers.Dense(N_CLASSES, activation=tf.nn.softmax)
])

# Compile the model with the specified loss function.
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Fit the model to the training data.
model.fit(x=input_dataset, epochs=10)

In [ ]:
from keras.utils import plot_model
model.summary()
plot_model(model, to_file='model_test.png',show_shapes=True)

## Check model accuracy on the test set

Now that we have a trained model, we can evaluate it using the test dataset.  To do that, read and prepare the test dataset in the same way as the training dataset.  Here we specify a batch size of 1 so that each example in the test set is used exactly once to compute model accuracy.  For model steps, just specify a number larger than the test dataset size (ignore the warning).

In [ ]:
test_dataset = (
  tf.data.TFRecordDataset(TEST_FILE_PATH, compression_type='GZIP')
    .map(parse_tfrecord, num_parallel_calls=5)
    .map(to_tuple)
    .batch(1))

model.evaluate(test_dataset, return_dict=True)
# input_dataset = parsed_dataset.map(to_tuple).batch(1)
# model.evaluate(input_dataset)

In [ ]:
model.fit(x=input_dataset, epochs=10)

In [ ]:
model_save_path = 'gs://zhangjian/model'
tf.keras.models.save_model(model,model_save_path)


In [ ]:
MODEL_DIR = 'gs://zhangjian/model'
model = tf.keras.models.load_model(MODEL_DIR)

# Use the trained model to classify an image from Earth Engine

Now it's time to classify the image that was exported from Earth Engine.  If the exported image is large, it will be split into multiple TFRecord files in its destination folder.  There will also be a JSON sidecar file called "the mixer" that describes the format and georeferencing of the image.  Here we will find the image files and the mixer file, getting some info out of the mixer that will be useful during model inference.

## Find the image files and JSON mixer file in Cloud Storage

Use `gsutil` to locate the files of interest in the output Cloud Storage bucket.  Check to make sure your image export task finished before running the following.

In [ ]:
# Get a list of all the files in the output bucket.
files_list = !gsutil ls 'gs://'{OUTPUT_BUCKET}
# Get only the files generated by the image export.
exported_files_list = [s for s in files_list if IMAGE_FILE_PREFIX in s]

# Get the list of image files and the JSON mixer file.
image_files_list = []
json_file = None
for f in exported_files_list:
  if f.endswith('.tfrecord.gz'):
    image_files_list.append(f)
  elif f.endswith('.json'):
    json_file = f

# Make sure the files are in the right order.
image_files_list.sort()

pprint(image_files_list)
print(json_file)

## Read the JSON mixer file

The mixer contains metadata and georeferencing information for the exported patches, each of which is in a different file.  Read the mixer to get some information needed for prediction.

In [ ]:
import json

# Load the contents of the mixer file to a JSON object.
json_text = !gsutil cat {json_file}
# Get a single string w/ newlines from the IPython.utils.text.SList
mixer = json.loads(json_text.nlstr)
pprint(mixer)

## Read the image files into a dataset

You can feed the list of files (`imageFilesList`) directly to the `TFRecordDataset` constructor to make a combined dataset on which to perform inference.  The input needs to be preprocessed differently than the training and testing.  Mainly, this is because the pixels are written into records as patches, we need to read the patches in as one big tensor (one patch for each band), then flatten them into lots of little tensors.

In [ ]:
# Get relevant info from the JSON mixer file.
patch_width = mixer['patchDimensions'][0]
patch_height = mixer['patchDimensions'][1]
patches = mixer['totalPatches']
patch_dimensions_flat = [patch_width * patch_height, 1]

# Note that the tensors are in the shape of a patch, one patch for each band.
image_columns = [
  tf.io.FixedLenFeature(shape=patch_dimensions_flat, dtype=tf.float32)
    for k in BANDS
]

# Parsing dictionary.
image_features_dict = dict(zip(BANDS, image_columns))

# Note that you can make one dataset from many files by specifying a list.
image_dataset = tf.data.TFRecordDataset(image_files_list, compression_type='GZIP')

# Parsing function.
def parse_image(example_proto):
  return tf.io.parse_single_example(example_proto, image_features_dict)

# Parse the data into tensors, one long tensor per patch.
image_dataset = image_dataset.map(parse_image, num_parallel_calls=5)

# Break our long tensors into many little ones.
image_dataset = image_dataset.flat_map(
  lambda features: tf.data.Dataset.from_tensor_slices(features)
)

# Add additional features (NDVI).
# image_dataset = image_dataset.map(
#   # Add NDVI to a feature that doesn't have a label.
#   lambda features: add_NDVI(features, None)[0]
# )

# Turn the dictionary in each record into a tuple without a label.
image_dataset = image_dataset.map(
  lambda data_dict: (tf.transpose(list(data_dict.values())), )
)

# Turn each patch into a batch.
image_dataset = image_dataset.batch(patch_width * patch_height)

## Generate predictions for the image pixels

To get predictions in each pixel, run the image dataset through the trained model using `model.predict()`.  Print the first prediction to see that the output is a list of the three class probabilities for each pixel.  Running all predictions might take a while.

In [ ]:
# Run prediction in batches, with as many steps as there are patches.
predictions = model.predict(image_dataset, steps=patches, verbose=1)

# Note that the predictions come as a numpy array.  Check the first one.
print(predictions[0])

## Write the predictions to a TFRecord file

Now that there's a list of class probabilities in `predictions`, it's time to write them back into a file, optionally including a class label which is simply the index of the maximum probability.  We'll write directly from TensorFlow to a file in the output Cloud Storage bucket.

Iterate over the list, compute class label and write the class and the probabilities in patches.  Specifically, we need to write the pixels into the file as patches in the same order they came out.  The records are written as serialized `tf.train.Example` protos.  This might take a while.

In [ ]:
print('Writing to file ' + OUTPUT_IMAGE_FILE)

In [ ]:
# Instantiate the writer.
writer = tf.io.TFRecordWriter(OUTPUT_IMAGE_FILE)

# Every patch-worth of predictions we'll dump an example into the output
# file with a single feature that holds our predictions. Since our predictions
# are already in the order of the exported data, the patches we create here
# will also be in the right order.
patch = [[], [], [], []]
cur_patch = 1
for prediction in predictions:
  patch[0].append(tf.argmax(prediction, 1))
  patch[1].append(prediction[0][0])
  patch[2].append(prediction[0][1])
  patch[3].append(prediction[0][2])
  # Once we've seen a patches-worth of class_ids...
  if (len(patch[0]) == patch_width * patch_height):
    print('Done with patch ' + str(cur_patch) + ' of ' + str(patches) + '...')
    # Create an example
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'prediction': tf.train.Feature(
              int64_list=tf.train.Int64List(
                  value=patch[0])),
          'bareProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[1])),
          'vegProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[2])),
          'waterProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[3])),
        }
      )
    )
    # Write the example to the file and clear our patch array so it's ready for
    # another batch of class ids
    writer.write(example.SerializeToString())
    patch = [[], [], [], []]
    cur_patch += 1

writer.close()

# Upload the classifications to an Earth Engine asset

## Verify the existence of the predictions file

At this stage, there should be a predictions TFRecord file sitting in the output Cloud Storage bucket.  Use the `gsutil` command to verify that the predictions image (and associated mixer JSON) exist and have non-zero size.

In [ ]:
!gsutil ls -l {OUTPUT_IMAGE_FILE}

## Upload the classified image to Earth Engine

Upload the image to Earth Engine directly from the Cloud Storage bucket with the [`earthengine` command](https://developers.google.com/earth-engine/command_line#upload).  Provide both the image TFRecord file and the JSON file as arguments to `earthengine upload`.

In [ ]:
print('Uploading to ' + OUTPUT_ASSET_ID)

In [ ]:
pip install oauth2client==3.0.0

In [ ]:
# Start the upload.
!earthengine upload image --asset_id={OUTPUT_ASSET_ID} --pyramiding_policy=mode {OUTPUT_IMAGE_FILE} {json_file}

## Check the status of the asset ingestion

You can also use the Earth Engine API to check the status of your asset upload.  It might take a while.  The upload of the image is an asset ingestion task.

In [ ]:
ee.batch.Task.list()

## View the ingested asset

Display the vector of class probabilities as an RGB image with colors corresponding to the probability of bare, vegetation, water in a pixel.  Also display the winning class using the same color palette.

In [ ]:
predictions_image = ee.Image(OUTPUT_ASSET_ID)

prediction_vis = {
  'bands': 'prediction',
  'min': 0,
  'max': 13,
  'palette': ['466b9f',
    'd1def8',
    'dec5c5',
    'd99282',
    'eb0000',
    'ab0000',
    'b3ac9f',
    '68ab5f',
    '1c5f2c']
}
probability_vis = {'bands': ['bareProb', 'vegProb', 'waterProb'], 'max': 0.5}

prediction_map_id = predictions_image.getMapId(prediction_vis)
probability_map_id = predictions_image.getMapId(probability_vis)

# map = folium.Map(location=[37.6413, -122.2582])
map = folium.Map(location=[-84.3242588445, 34.252569898362])
folium.TileLayer(
  tiles=prediction_map_id['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='prediction1',
).add_to(map)
folium.TileLayer(
  tiles=probability_map_id['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='probability1',
).add_to(map)
map.add_child(folium.LayerControl())
map